# Web Scraping

Nombre: Marcela Cabrera

In [10]:
import requests
from bs4 import BeautifulSoup
import json

In [12]:
HEADERS = {
    "User-Agent": "Mozilla/5.0"
}
def get_recipe(url):
    response = requests.get(url, headers=HEADERS, timeout=10)
    soup = BeautifulSoup(response.text, "html.parser")

    recipe_data = None

    # Buscar datos JSON-LD tipo Recipe
    for script in soup.find_all("script", type="application/ld+json"):
        try:
            data = json.loads(script.string)
            items = data if isinstance(data, list) else [data]

            for item in items:
                recipe_type = item.get("@type")
                if recipe_type == "Recipe" or (
                    isinstance(recipe_type, list) and "Recipe" in recipe_type
                ):
                    recipe_data = item
                    break
        except Exception:
            continue

        if recipe_data:
            break

    if not recipe_data:
        return None

    return {
        "url": url,
        "puntaje": recipe_data.get("aggregateRating", {}).get("ratingValue"),
        "descripcion": recipe_data.get("description"),
        "prep_time": recipe_data.get("prepTime"),
        "cook_time": recipe_data.get("cookTime"),
        "total_time": recipe_data.get("totalTime"),
        "porciones": recipe_data.get("recipeYield"),
        "ingredientes": recipe_data.get("recipeIngredient", []),
        "pasos": [
            step.get("text")
            for step in recipe_data.get("recipeInstructions", [])
            if isinstance(step, dict)
        ],
        "nutricion": recipe_data.get("nutrition")
    }


In [14]:
urls = [
    "https://www.allrecipes.com/recipe/9999/peanut-butter-bars-i/",
    "https://www.allrecipes.com/grill-master-chicken-wing-recipe-11885412",
    "https://www.allrecipes.com/million-dollar-soup-recipe-11823470",
    "https://www.allrecipes.com/recipe/270770/garlic-noodles/",
    "https://www.allrecipes.com/chicken-cobbler-recipe-7966464",
    "https://www.allrecipes.com/recipe/235997/unstuffed-cabbage-roll/"
]


In [15]:
recetas = []

for url in urls:
    receta = get_recipe(url)
    if receta:
        recetas.append(receta)

print("Total recetas obtenidas:", len(recetas))

Total recetas obtenidas: 5


In [16]:
for i, r in enumerate(recetas, 1):
    print("\n" + "=" * 60)
    print(f"RECETA {i}")
    print("URL:", r["url"])
    print("PUNTAJE:", r["puntaje"])
    print("DESCRIPCIÓN:", r["descripcion"])
    print("PREP TIME:", r["prep_time"])
    print("COOK TIME:", r["cook_time"])
    print("TOTAL TIME:", r["total_time"])
    print("PORCIONES:", r["porciones"])

    print("\nINGREDIENTES:")
    for ing in r["ingredientes"]:
        print("-", ing)

    print("\nPASOS:")
    for n, paso in enumerate(r["pasos"], 1):
        print(f"{n}. {paso}")

    print("\nNUTRICIÓN:")
    if r["nutricion"]:
        for k, v in r["nutricion"].items():
            print(f"{k}: {v}")
    else:
        print("No disponible")


RECETA 1
URL: https://www.allrecipes.com/recipe/9999/peanut-butter-bars-i/
PUNTAJE: 4.6
DESCRIPCIÓN: These peanut butter bars are rich, creamy, and taste just like peanut butter cups. Made with just 5 ingredients, these no-bake bars are easy to make.
PREP TIME: PT25M
COOK TIME: None
TOTAL TIME: PT85M
PORCIONES: ['12', '12 bars']

INGREDIENTES:
- 2 cups graham cracker crumbs
- 2 cups confectioners' sugar
- 1 cup butter or margarine, melted
- 1 cup peanut butter
- 1.5 cups semisweet chocolate chips
- 4 tablespoons peanut butter

PASOS:
1. Gather all ingredients.
2. Mix together graham cracker crumbs, confectioners&#39; sugar, butter or margarine, and 1 cup peanut butter in a medium bowl until well-blended.
3. Press evenly into the bottom of an ungreased 9x13-inch pan.
4. Place chocolate chips and 4 tablespoons peanut butter in a microwave-safe bowl. Microwave on high, stirring every 15 seconds, until smooth.
5. Spread mixture over crust.
6. Refrigerate for at least 1 hour before cutting